In [1]:
from models import *
from utils.utils import *
import numpy as np
from copy import deepcopy
from test import test
from terminaltables import AsciiTable
import time
from utils.prune_utils import *
import argparse

import warnings
warnings.filterwarnings("ignore")

In [2]:
img_size = 800
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cfg1 = 'cfg/dense_yolov3_4.cfg'
cfg2 = 'cfg/prune_10_shortcut_prune_0.8_keep_0.01_dense_yolov3_4.cfg'

weights1 = 'weights/sparse_421map_best.pt'
weights2 = 'weights/last.pt'

model1 = Darknet(cfg1, (img_size, img_size)).to(device)
model2 = Darknet(cfg2, (img_size, img_size)).to(device)


        
if weights1.endswith(".pt"):
    model1.load_state_dict(torch.load(weights1, map_location=device)['model'])
else:
    _ = load_darknet_weights(model1, weights1)
print('\nloaded weights from ',weights1)

if weights2.endswith(".pt"):
    model2.load_state_dict(torch.load(weights2, map_location=device)['model'])
else:
    _ = load_darknet_weights(model2, weights2)
print('\nloaded weights from ',weights2)


Model Summary: 254 layers, 6.19432e+07 parameters, 6.19432e+07 gradients
Model Summary: 194 layers, 2.42303e+06 parameters, 2.42303e+06 gradients

loaded weights from  weights/sparse_421map_best.pt

loaded weights from  weights/last.pt


In [3]:
random_input = torch.rand((1, 3, img_size, img_size)).to(device)

def obtain_avg_forward_time(input, model, repeat=200):

    model.eval()
    start = time.time()
    with torch.no_grad():
        for i in range(repeat):
            output = model(input)
    avg_infer_time = (time.time() - start) / repeat

    return avg_infer_time, output

model1_forward_time, _ = obtain_avg_forward_time(random_input, model1)
model2_forward_time, _ = obtain_avg_forward_time(random_input, model2)



eval_model = lambda model,cfg:test(model = model, cfg= cfg, data='data/visdrone.data',batch_size=64, imgsz=img_size,is_training = False)
obtain_num_parameters = lambda model:sum([param.nelement() for param in model.parameters()])

print("\nlet's test the original model first:")
with torch.no_grad():
    origin_model_metric1 = eval_model(model1,cfg1)
    origin_model_metric2 = eval_model(model2,cfg2)
origin_nparameters1 = obtain_num_parameters(model1)
origin_nparameters2 = obtain_num_parameters(model2)

Caching labels /home/tione/notebook/seu/VisDrone2019-DET-val/labels.txt (548 found, 0 missing, 0 empty, 0 duplicate, for 548 images): 100%|██████████| 548/548 [00:00<00:00, 3370.98it/s]


let's test the original model first:



               Class    Images   Targets         P         R   mAP@0.5        F1: 100%|██████████| 9/9 [01:46<00:00, 11.80s/it]
Caching labels /home/tione/notebook/seu/VisDrone2019-DET-val/labels.txt (548 found, 0 missing, 0 empty, 0 duplicate, for 548 images): 100%|██████████| 548/548 [00:00<00:00, 3361.14it/s]
               Class    Images   Targets         P         R   mAP@0.5        F1:   0%|          | 0/9 [00:00<?, ?it/s]

                 all       548  3.88e+04     0.398     0.484     0.422     0.435


               Class    Images   Targets         P         R   mAP@0.5        F1: 100%|██████████| 9/9 [01:35<00:00, 10.57s/it]


                 all       548  3.88e+04     0.439     0.524      0.46     0.477


In [4]:
metric_table = [
    ["Metric", "Before Prune", "After Prune"],
    ["mAP", f'{origin_model_metric1[0][2]:.6f}', f'{origin_model_metric2[0][2]:.6f}'],
    ["Parameters", f"{origin_nparameters1}", f"{origin_nparameters2}"],
    ["Inference time", f'{model1_forward_time:.6f}', f'{model2_forward_time:.6f}']
]
print(AsciiTable(metric_table).table)

+------------+--------------+-------------+
| Metric     | Before Prune | After Prune |
+------------+--------------+-------------+
| mAP        | 0.421684     | 0.460239    |
| Parameters | 61943156     | 2423033     |
| Inference  | 0.025589     | 0.009889    |
+------------+--------------+-------------+
